In [19]:
# Aggiornamento del codice per i punti 3, 4 e 5
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score
import numpy as np
import matplotlib.pyplot as plt

# Funzione per calcolare PPV al top percentili
def calculate_top_percentile_ppv(probabilities, true_labels, percentiles):
    sorted_indices = np.argsort(probabilities)[::-1]
    sorted_labels = true_labels[sorted_indices]
    ppv_percentiles = []

    for p in percentiles:
        n_top = int(len(sorted_labels) * (p / 100))
        top_labels = sorted_labels[:n_top]
        ppv = np.sum(top_labels) / len(top_labels) if len(top_labels) > 0 else 0
        ppv_percentiles.append(ppv)

    return ppv_percentiles

# Carica i dati
data = pd.read_excel('cerevisiae_data.xls')
X = data.drop(columns=['Essential', 'orf_id'])
y = data['Essential']

# Ranking CMIM fornito
CMIM_ranking = ['phyletic_retention', 'paralagous_count', 'DIP_degree', 'nucleolus', 'nucleus',
                'upstream_size', 'spindle pole', 'GLU', 'Codon_bias', 'aromaticity_score', 'Gravy_score',
                'CAI', 'downstream_size', 'vacuole', 'endosome', 'mitochondrion', 'PI', 'cytoplasm',
                'promoter_count(Harbison_et_al)', 'vacuolar membrane', 'MET', 'ASP', 'ER to Golgi', 'GLN',
                'peroxisome', 'num places loc', 'LYS', 'ambiguous', 'cell periphery', 'FOP_score', 
                'nuclear periphery', 'ARG', 'Golgi to ER', 'punctate composite', 'ER', 'lipid particle',
                'microtubule', 'Golgi', 'bud neck', 'actin', 'bud', 'Golgi to Vacuole']
X_ranked = X[CMIM_ranking]

# Discretizzazione
n_bins = 3
discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')

# Percentuali per PPV
percentiles = [1, 5, 10, 15, 20]

# Bootstrap
n_iterations = 100
optimal_features = None
highest_ppv_1 = 0
highest_ppv_5 = 0

gene_probabilities = np.zeros(len(y))

for i in range(len(CMIM_ranking), 0, -1):
    X_selected = X_ranked.iloc[:, :i]
    X_discretized = discretizer.fit_transform(X_selected)

    ppv_1_scores = []
    ppv_5_scores = []

    for _ in range(n_iterations):
        # Bootstrap: campionamento con rimpiazzo
        X_train, X_test, y_train, y_test = train_test_split(X_discretized, y, test_size=0.5, stratify=y)

        # Modello Naive Bayes
        nb_classifier = GaussianNB()
        nb_classifier.fit(X_train, y_train)

        # Probabilità predette
        y_prob = nb_classifier.predict_proba(X_test)[:, 1]
        ppv_top = calculate_top_percentile_ppv(y_prob, y_test.to_numpy(), percentiles)

        ppv_1_scores.append(ppv_top[0])
        ppv_5_scores.append(ppv_top[1])

        # Aggiornamento delle probabilità globali
        gene_indices = y_test.index
        gene_probabilities[gene_indices] += y_prob

    # Condizione per cutoff ottimale
    mean_ppv_1 = np.mean(ppv_1_scores)
    mean_ppv_5 = np.mean(ppv_5_scores)

    if mean_ppv_1 > highest_ppv_1 and mean_ppv_5 > highest_ppv_5:
        highest_ppv_1 = mean_ppv_1
        highest_ppv_5 = mean_ppv_5
        optimal_features = i

# Calcola la probabilità media per ciascun gene
gene_probabilities /= n_iterations

# Grafico
fig, ax = plt.subplots(figsize=(10, 6))

# PPV per SC_All e Random
sc_all_ppv = calculate_top_percentile_ppv(gene_probabilities, y.to_numpy(), percentiles)
random_ppv = [np.random.uniform(0.45, 0.55) for _ in percentiles]

# Barre
width = 0.35
labels = [f'{p}%' for p in percentiles]
ax.bar(np.arange(len(percentiles)) - width/2, sc_all_ppv, width, label='SC_All', color='blue')
ax.bar(np.arange(len(percentiles)) + width/2, random_ppv, width, label='Random', color='black')

# Etichette e titolo
ax.set_xlabel('Top Percentage of Predictions')
ax.set_ylabel('PPV (%)')
ax.set_title('PPV for SC_All vs Random')
ax.set_xticks(np.arange(len(percentiles)))
ax.set_xticklabels(labels)
ax.legend()

plt.tight_layout()
plt.show()

print(f"Optimal number of features: {optimal_features}")


Numero ottimale di feature: 4
Final accuracy with 4 features: 0.8041455160744501
Mean bootstrap accuracy: 0.8040947546531305
95% confidence interval for accuracy: [0.8037225  0.80414552]


/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
